<div class="alert alert-block alert-success">

# **1.** Environment Setup

<div>

In [1]:
from google.colab import drive, files

In [2]:
import os
import sys

In [3]:
is_colab = 'google.colab' in sys.modules

if is_colab:
    base_path = "/content/drive/.shortcut-targets-by-id/1hNB4s6RR7JeKfFdGwm27WrF6pNXXIg3R/Deep Learning"
    drive.mount('/content/drive')

    os.chdir('/content/drive/MyDrive/Deep Learning')
    print("Changed directory to:", os.getcwd())
    !pip install keras_cv
    !pip install -q -U keras-tuner
else:
    base_path = ".."

Mounted at /content/drive
Changed directory to: /content/drive/.shortcut-targets-by-id/1hNB4s6RR7JeKfFdGwm27WrF6pNXXIg3R/Deep Learning
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.7 MB/s eta 0:00:00


## 1.1 Import Libraries

In [4]:
import pandas as pd
import zipfile
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import regularizers
from classes import *
from functions import *

In [5]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.metrics import AUC, F1Score, CategoricalAccuracy, TopKCategoricalAccuracy
from sklearn.metrics import f1_score, precision_score, recall_score
from keras import optimizers

In [6]:
#Load the DataFrames from the .pkl files
with open(os.path.join(base_path,"data/train_df.pkl"), "rb") as f:
     train_df = pickle.load(f)

with open(os.path.join(base_path,"data/valid_df.pkl"), "rb") as f:
     val_df = pickle.load(f)

with open(os.path.join(base_path,"data/test_df.pkl"), "rb") as f:
     test_df = pickle.load(f)

with open(os.path.join(base_path,"data/train_df_sampled.pkl"), "rb") as f:
     train_df_sampled = pickle.load(f)

with open(os.path.join(base_path,"data/family_encoder.pkl"), "rb") as f:
     family_encoder = pickle.load(f)

In [7]:
class_names = family_encoder.classes_

class_names

array(['accipitridae', 'acipenseridae', 'acroporidae', 'agamidae',
       'agariciidae', 'albulidae', 'alcedinidae', 'alligatoridae',
       'alopiidae', 'ambystomatidae', 'anatidae', 'anguidae', 'aotidae',
       'apidae', 'ardeidae', 'arthroleptidae', 'atelidae', 'attelabidae',
       'balaenicipitidae', 'balaenidae', 'balaenopteridae', 'balistidae',
       'bombycillidae', 'bovidae', 'brachypteraciidae', 'bucerotidae',
       'bufonidae', 'burhinidae', 'cacatuidae', 'callitrichidae',
       'callorhinchidae', 'caprimulgidae', 'carabidae', 'carcharhinidae',
       'cardiidae', 'carettochelyidae', 'cebidae', 'cerambycidae',
       'cercopithecidae', 'cervidae', 'cetorhinidae', 'chaetodontidae',
       'chamaeleonidae', 'charadriidae', 'cheirogaleidae', 'chelidae',
       'cheloniidae', 'chelydridae', 'ciconiidae', 'coenagrionidae',
       'colubridae', 'columbidae', 'conidae', 'cracidae', 'cricetidae',
       'crocodylidae', 'cryptobranchidae', 'ctenomyidae', 'cuculidae',
       'cypr

<div class="alert alert-block alert-success">

# **2.** Preprocessing

<div>

In [ ]:
with open("../data/train_df.pkl", "rb") as f:
     train_df = pickle.load(f)

with open("../data/val_df.pkl", "rb") as f:
     val_df = pickle.load(f)

with open("../data/test_df.pkl", "rb") as f:
     test_df = pickle.load(f)

In [ ]:
with open('family_encoder.pkl', 'rb') as f:
    family_encoder = pickle.load(f)

# Load class names
class_names = family_encoder.classes_

class_names

In [8]:
minority_class = train_df['family'].value_counts()[train_df['family'].value_counts() < 25].index
minority_class=minority_class.to_list()

In [9]:
batch_size = 32 ## the less the better because in each epoch the model sees N / batch_size images
image_size = (224, 224)

preprocess = Preprocessor(image_size=image_size, batch_size=batch_size)

<div class="alert alert-block alert-success">

# **3.** Results Analysis

<div>

In [10]:
# Load model keras
model = load_model(os.path.join(base_path,"models/efficient_net_finetuned_final.keras"))

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 34 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
# load datasets with efficientnets preprocessing
train_ds, _ = preprocess.load_img(
    data_dir=os.path.join(base_path,"data/rare_species/train"),
    minority_class=minority_class,
    augment="mixup",
    oversampling=True,
    shuffle= True,
    preprocessing_function=preprocess_input)

val_ds, _ = preprocess.load_img(
    data_dir=os.path.join(base_path,"data/rare_species/val"),
    minority_class=[],
    augment=None,
    shuffle= False,
    preprocessing_function=preprocess_input)

test_ds, _ = preprocess.load_img(
    data_dir=os.path.join(base_path,"data/rare_species/test"),
    minority_class=[],
    augment=None,
    shuffle= False,
    preprocessing_function=preprocess_input)

Found 8388 files belonging to 202 classes.
Found 1797 files belonging to 202 classes.
Found 1798 files belonging to 202 classes.


In [12]:
y_pred = model.predict(val_ds)

57/57 ━━━━━━━━━━━━━━━━━━━━ 389s 7s/step


### Classification report

In [13]:
def get_metric(dataset, model_name):
    print(f"\nEvaluating model: {model_name}")
    model = load_model(model_name)

    y_true = []
    y_pred = []

    for batch_x, batch_y in dataset:
        preds = model.predict(batch_x, verbose=0)
        y_true.append(np.argmax(batch_y.numpy(), axis=1))
        y_pred.append(np.argmax(preds, axis=1))

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    print(classification_report(y_true, y_pred, digits=4))
    print("Accuracy     :", accuracy_score(y_true, y_pred))
    print("F1 (macro)   :", f1_score(y_true, y_pred, average="macro"))
    print("Precision    :", precision_score(y_true, y_pred, average="macro"))
    print("Recall       :", recall_score(y_true, y_pred, average="macro"))

    return y_true, y_pred

In [ ]:
get_metric(train_ds, os.path.join(base_path,"models/efficient_net_finetuned_final.keras"))


Evaluating model: /content/drive/.shortcut-targets-by-id/1hNB4s6RR7JeKfFdGwm27WrF6pNXXIg3R/Deep Learning/models/efficient_net_finetuned_final.keras


In [ ]:
get_metric(val_ds, os.path.join(base_path,"models/efficient_net_finetuned_final.keras"))

### Visualize no confidence images

In [1]:
# Unbatch train_ds into list
train_images = []
train_labels = []

for img, label in train_ds.unbatch():
    train_images.append(img.numpy())
    train_labels.append(label.numpy())

train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Predict all train images
pred_probs_all = model.predict(train_images, verbose=1)
y_pred = np.argmax(pred_probs_all, axis=1)
y_true = train_labels

NameError: name 'train_ds' is not defined

In [ ]:
train_labels.shape

In [ ]:
train_images

In [ ]:
pred_probs_all

In [ ]:
np.max(pred_probs_all, axis=1)

In [ ]:
# Plot histogram of certainties
plt.figure(figsize=(8, 5))
plt.hist(np.max(pred_probs_all, axis=1), bins=30, edgecolor='black')
plt.xlabel('Model Certainty (Max Softmax Probability)')
plt.ylabel('Number of Samples')
plt.title('Distribution of Model Certainty')
plt.grid(True)
plt.show()

#### Histogram of model confidence, divided by correctly and incorrectly classified images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- assume pred_probs_all and train_labels are already defined ---

# 1) prepare labels
labels = train_labels
if labels.ndim > 1:
    labels = np.argmax(labels, axis=1)

# 2) get preds and confidences
preds = np.argmax(pred_probs_all, axis=1)
confidences = np.max(pred_probs_all, axis=1)

# 3) split confidences by correctness
correct_conf = confidences[preds == labels]
incorrect_conf = confidences[preds != labels]

# 4) define bins
bins = np.linspace(0, 1, 21)  # 20 bins

# 5) histogram counts for each bin
correct_counts, _   = np.histogram(correct_conf,   bins=bins)
incorrect_counts, _ = np.histogram(incorrect_conf, bins=bins)
total_counts = correct_counts + incorrect_counts

# 6) misclassification rate per bin (in %), guard against zero-division
with np.errstate(divide='ignore', invalid='ignore'):
    misclass_rate = 100 * incorrect_counts / total_counts
misclass_rate = np.nan_to_num(misclass_rate)  # zero where total_counts==0

# 7) bin centers for plotting the line
bin_centers = (bins[:-1] + bins[1:]) / 2

# 8) plot
fig, ax1 = plt.subplots(figsize=(8, 5))

# stacked histogram
ax1.hist(
    [correct_conf, incorrect_conf],
    bins=bins,
    stacked=True,
    color=['green', 'red'],
    label=['Correct classification', 'Incorrect classification'],
    edgecolor="black"
)
ax1.set_xlabel('Model Confidence (Max Softmax Probability)')
ax1.set_ylabel('Number of Samples')
ax1.legend(loc='upper left')

# secondary axis for misclassification %
ax2 = ax1.twinx()
ax2.plot(
    bin_centers,
    misclass_rate,
    marker='o',
    linestyle='-',
    color='blue',
    label='Misclassification rate (%)'
)
ax2.set_ylabel('Misclassification Rate (%)')
ax2.set_ylim(0, 100)
ax2.legend(loc='upper right')

plt.title('Distribution of Model Confidence')
plt.tight_layout()
plt.show()


#### Trying to incorporate phylum

In [ ]:
# Unbatch train_ds into list
val_images = []
val_labels = []

for img, label in val_ds.unbatch():
    val_images.append(img.numpy())
    val_labels.append(label.numpy())

val_images = np.array(val_images)
val_labels = np.array(val_labels)

# Predict all val images
pred_probs_all_val = model.predict(val_images, verbose=1)
y_pred = np.argmax(pred_probs_all, axis=1)
y_true = val_labels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# create family:phylum dicionary
meta = pd.read_csv('../data/rare_species/metadata.csv')
dup = meta.groupby('family')['phylum'].nunique()
family_to_phylum = dict(zip(meta['family'], meta['phylum']))


In [ ]:
# --- 2) Prepare true labels & names ---
# train_labels: your true labels, shape (11200,) or one-hot (11200,202)
# class_names: list of length 202 mapping index→family string
labels = np.argmax(val_labels, axis=1) # family# prediction, not one-hot-encoded
y_true_fam   = np.array([class_names[i] for i in labels])
y_true_phyl  = np.array([family_to_phylum[f] for f in y_true_fam])


In [ ]:
# --- 3) Baseline predictions & phylum check ---
y_pred       = np.argmax(pred_probs_all_val, axis=1)
y_pred_fam   = np.array([class_names[i] for i in y_pred])
y_pred_phyl  = np.array([family_to_phylum[f] for f in y_pred_fam])

In [ ]:
# a dictionary phylum:[list of indices of classes that belong to that phylum]
phylum_to_inds = {}
for idx, fam in enumerate(class_names):
    ph = family_to_phylum[fam]
    phylum_to_inds.setdefault(ph, []).append(idx)


In [ ]:
# --- 4) Hierarchical override logic ---
y_pred_hier = y_pred.copy()
for i, probs in enumerate(pred_probs_all_val):
    # if predicted phylum ≠ true phylum, force to best within true phylum
    if y_pred_phyl[i] != y_true_phyl[i]:
        valid_idx = phylum_to_inds[y_true_phyl[i]]
        y_pred_hier[i] = valid_idx[np.argmax(probs[valid_idx])]



In [ ]:
# --- 5) Overall accuracy comparison ---
acc_base  = accuracy_score(labels, y_pred)
acc_hier  = accuracy_score(labels, y_pred_hier)
print(f"Baseline overall accuracy:     {acc_base:.4f}")
print(f"Hierarchical override accuracy:{acc_hier:.4f}")

In [ ]:
print("Classification report with original predictions:")
print(classification_report(labels,y_pred))
print("Classification report with original predictions:")
print(classification_report(labels,y_pred_hier,digits=4))

In [ ]:
# --- 6) Per-phylum accuracy improvement ---
phyla = np.unique(y_true_phyl)
base_scores = []
hier_scores = []
for ph in phyla:
    idxs = np.where(y_true_phyl == ph)[0]
    base_scores.append( accuracy_score(labels[idxs], y_pred[idxs]) )
    hier_scores.append( accuracy_score(labels[idxs], y_pred_hier[idxs]) )

x = np.arange(len(phyla))
width = 0.35

plt.figure(figsize=(10,5))
plt.bar(x - width/2, base_scores, width, label='Baseline')
plt.bar(x + width/2, hier_scores, width, label='Hierarchical')
plt.xticks(x, phyla, rotation=45, ha='right')
plt.ylabel('Accuracy')
plt.title('Accuracy by Phylum')
plt.legend()
plt.tight_layout()
plt.show()


#### No certainty

In [ ]:
# Convert y_true to class labels
y_true_labels = np.argmax(y_true, axis=1)

# Get misclassified indices
misclassified_indices = np.where(y_true_labels != y_pred)[0]

# For each misclassified, get model confidence (highest softmax probability)
confidences = np.max(pred_probs_all[misclassified_indices], axis=1)

# Sort misclassified examples by descending confidence
sorted_indices = np.argsort(confidences)  # Sort ascending
selected_indices = misclassified_indices[sorted_indices[:12]]  # Take top 6

# Prepare images to show (selected misclassified ones)
images_to_show = []

for idx in selected_indices:
    img = train_images[idx]
    true_label = y_true_labels[idx]
    pred_label = y_pred[idx]
    confidence = confidences[np.where(misclassified_indices == idx)][0]  # Get confidence for the current misclassified image
    images_to_show.append((img, true_label, pred_label, confidence))

# Save the misclassified data
misclassified_data = {
    "True Label": [class_names[true_label] for _, true_label, _, _ in images_to_show],
    "Predicted Label": [class_names[pred_label] for _, _, pred_label, _ in images_to_show],
    "Confidence": [confidence for _, _, _, confidence in images_to_show]
}

# Plot
plt.figure(figsize=(16, 10))

for i, (img, true_label, pred_label, confidence) in enumerate(images_to_show):
    plt.subplot(4, 3, i + 1)
    plt.imshow(img.astype("uint8"))
    plt.axis('off')
    plt.title(f"True: {class_names[true_label]}\nPred: {class_names[pred_label]}\nConf: {confidence:.2f}",
              color='red', fontsize=10)

plt.suptitle("Less Confident Misclassifications", fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:
# Plot histogram of certainties
plt.figure(figsize=(8, 5))
plt.hist(confidences, bins=30, edgecolor='black')
plt.xlabel('Model Certainty (Max Softmax Probability)')
plt.ylabel('Number of Misclassified Samples')
plt.title('Distribution of Model Certainty for Misclassified Samples')
plt.grid(True)
plt.show()

Since you have 202 classes, a model that is completely uncertain (i.e., guessing randomly) would have maximum softmax probability 0.004. But models aren't truly random even when uncertain — usually, even bad predictions are around 0.02–0.1 certainty (depends on how the softmax behaves).

In [ ]:
# First, compute the maximum probability for ALL predictions
all_confidences = np.max(pred_probs_all, axis=1)  # Shape: (num_val_samples,)

# Total number of validation images
total_val_images = len(train_images)

# Number of predictions where certainty < 0.1
num_uncertain_predictions = np.sum(all_confidences < 0.019)

print(f"Total validation images: {total_val_images}")
print(f"Number of predictions with certainty < 0.1: {num_uncertain_predictions}")
print(f"Percentage: {100 * num_uncertain_predictions / total_val_images:.2f}%")

In [ ]:
misclassified_data

#### Top Correctly Classified

In [ ]:
# Convert y_true to class labels
y_true_labels = np.argmax(y_true, axis=1)

# Get indices of correctly classified images
correct_indices = np.where(y_true_labels == y_pred)[0]

# For each correctly classified image, get model confidence (highest softmax probability)
confidences = np.max(pred_probs_all[correct_indices], axis=1)

# Sort correctly classified examples by descending confidence
sorted_indices = np.argsort(confidences)[::-1]  # Sort descending
selected_indices = correct_indices[sorted_indices[:6]]  # Take top 6 with highest confidence

# Prepare images to show (selected correctly classified ones)
images_to_show = []

for idx in selected_indices:
    img = test_images[idx]
    true_label = y_true_labels[idx]
    pred_label = y_pred[idx]
    confidence = confidences[np.where(correct_indices == idx)][0]  # Get confidence for the current correctly classified image
    images_to_show.append((img, true_label, pred_label, confidence))

# Save the misclassified data
correctly_classified_data = {
    "True Label": [class_names[true_label] for _, true_label, _, _ in images_to_show],
    "Predicted Label": [class_names[pred_label] for _, _, pred_label, _ in images_to_show],
    "Confidence": [confidence for _, _, _, confidence in images_to_show]
}

# Plot
plt.figure(figsize=(16, 10))

for i, (img, true_label, pred_label, confidence) in enumerate(images_to_show):
    plt.subplot(2, 3, i + 1)
    plt.imshow(img.astype("uint8"))
    plt.axis('off')
    plt.title(f"True: {class_names[true_label]}\nPred: {class_names[pred_label]}\nConf: {confidence:.2f}",
              color='green', fontsize=10)

plt.suptitle("Most Confident Correct Classifications", fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:
correctly_classified_data

#### Correctly Classified for Specific Class

In [ ]:
misclassified_data

In [ ]:
for family in misclassified_data["True Label"]:
    show_correct_predictions_for_family(y_true, y_pred, pred_probs_all, test_images, class_names, family_name=family, num_images=6)

In [ ]:
for family in misclassified_data["Predicted Label"]:
    show_correct_predictions_for_family(y_true, y_pred, pred_probs_all, test_images, class_names, family_name=family, num_images=6)

In [ ]:
correctly_classified_data

In [ ]:
for family in correctly_classified_data["True Label"]:
    show_correct_predictions_for_family(y_true, y_pred, pred_probs_all, test_images, class_names, family_name=family, num_images=6)

<div class="alert alert-block alert-success">

# **4.** More

<div>

In [ ]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt


In [ ]:
files = {
    "VGG16": "vgg_history_train_until_opt.csv",
    "ResNet50": "resnet50_final_train_history.csv",
    "EfficientNet Baseline 1": "efficient_net_baseline1_history.csv",
    "EfficientNet Baseline 2": "efficient_net_baseline2_history.csv",
    "EfficientNet Final": "efficient_net_final_train_history.csv",
    "EfficientNet Fine-Tuned": "efficient_net_final_finetune_history.csv"
}


In [ ]:
def load_history(path):
    df = pd.read_csv(path)
    if 'acc' in df.columns:
        df = df.rename(columns={'acc': 'accuracy'})
    if 'val_acc' in df.columns:
        df = df.rename(columns={'val_acc': 'val_accuracy'})
    return df


In [ ]:
histories = {label: load_history(path) for label, path in files.items()}

summary_list = []
for label, df in histories.items():
    last_epoch = df.iloc[-1]
    # Para cada métrica, tenta pegar 'accuracy' e 'f1_score'
    summary_list.append({
        "Model": label,
        "Train Set Accuracy": last_epoch.get("accuracy", last_epoch.get("accuracy", None)),
        "Validation Set Accuracy": last_epoch.get("val_accuracy", last_epoch.get("val_accuracy", None)),
        "Train Set F1-Score": last_epoch.get("f1_score", last_epoch.get("f1_score", None)),
        "Validation Set F1-Score": last_epoch.get("val_f1_score", last_epoch.get("val_f1_score", None)),
        "Train Set Loss": last_epoch.get("loss", last_epoch.get("loss", None)),
        "Validation Set Loss": last_epoch.get("val_loss", last_epoch.get("val_loss", None))
    })

summary_df = pd.DataFrame(summary_list)
summary_df = summary_df.set_index("Model")
summary_df = summary_df.sort_values(by="Validation Set F1-Score", ascending=False)


### Final Summary

In [ ]:
summary_df = summary_df.sort_values(by="Validation Set F1-Score", ascending=False)

In [ ]:
summary_df

### Plots

In [ ]:
plt.rcParams['font.family'] = 'Aptos'

In [ ]:
sns.set_style("white")
plt.figure(figsize=(8, 10))
ax = summary_df[['Train Set Accuracy', 'Validation Set Accuracy']].plot(kind='bar', color=['navy', 'maroon'], legend=False)
ax.legend(loc='upper right', fontsize=10)
plt.ylabel("Score")
plt.ylim(0, 1)
plt.xticks(rotation=90)
plt.show()


In [ ]:
sns.set_style("white")
plt.figure(figsize=(8, 10))
ax = summary_df[['Train Set Loss', 'Validation Set Loss']].plot(kind='bar', color=['navy', 'maroon'], legend=False)
ax.legend(loc='upper left', fontsize=10)
plt.ylabel("Loss")
plt.ylim(0, 2.5)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.set_style("white")
plt.figure(figsize=(8, 10))
ax = summary_df[['Train Set F1-Score', 'Validation Set F1-Score']].plot(kind='bar', color=['navy', 'maroon'], legend=False)
ax.legend(loc='upper right', fontsize=10)
plt.ylabel("F1-Score")
plt.ylim(0, 1)
plt.xticks(rotation=90)
plt.show()

### Test Performance

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [ ]:
test_ds, _ = preprocess.load_img(
    data_dir="../data/rare_species/test",
    minority_class=[],
    augment=None,
    shuffle= False,
    preprocessing_function=preprocess_input)

In [ ]:
get_metric(test_ds, "efficient_net_finetuned_final.keras")